In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

model_name = "bert-base-uncased"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad(): # we are not training the model, so we don't need gradients
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
    return embedding




In [5]:
def calculate_cosine_similarity(emb_title1, emb_title2):
    emb_title1 = emb_title1.reshape(1, -1)
    emb_title2 = emb_title2.reshape(1, -1)

    similarity = cosine_similarity(emb_title1, emb_title2)[0][0]
    return similarity


In [ ]:
import ast
with open('all_distance_pairs.txt', 'r') as f:
    all_distance_pairs = ast.literal_eval(f.read())
with open('all_articles.txt', 'r') as f:
    all_articles = ast.literal_eval(f.read())

# get the embeddings of all articles and store them in a dictionary
article_embeddings = {}
for article in all_articles:
    article_embeddings[article] = get_embedding(article)

# calculate the cosine similarity between all pairs of articles
similarities = {}
for pair in all_distance_pairs:
    title1 = pair[0]
    title2 = pair[1]
    embedding1 = article_embeddings[title1]
    embedding2 = article_embeddings[title2]
    similarity =  calculate_cosine_similarity(embedding1, embedding2)
    similarities[(title1, title2)] = similarity


In [15]:
# get the format of the dataframes
import pandas as pd
df = pd.DataFrame(similarities.items(), columns=['pair', 'cosine_similarity'])

In [8]:
%pip install sentence_transformers

Note: you may need to restart the kernel to use updated packages.


In [12]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

def calculate_sbert_similarity(title1, title2):
    # Get embeddings
    embedding1 = model.encode(title1, convert_to_tensor=True)
    embedding2 = model.encode(title2, convert_to_tensor=True)
    # Calculate cosine similarity using SBERT's util function
    similarity = util.pytorch_cos_sim(embedding1, embedding2).item()
    return similarity



In [ ]:
#using the SBERT model to calculate the similarity between all pairs of articles by adding a coloumn to the dataframe similarities
df['sbert_similarity'] = df['pair'].apply(lambda x: calculate_sbert_similarity(x[0], x[1]))

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from scipy.spatial.distance import euclidean

# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"  # You can use any suitable model here
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
    # Tokenize the text and get embeddings
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    # Pool the output to get a single vector representation of the text
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
    return embedding

def calculate_euclidean_distance(title1, title2):
    # Get embeddings
    embedding1 = get_embedding(title1)
    embedding2 = get_embedding(title2)
    # Convert to numpy arrays for distance calculation
    embedding1_np = embedding1.numpy()
    embedding2_np = embedding2.numpy()
    # Calculate Euclidean distance
    distance = euclidean(embedding1_np, embedding2_np)
    return distance

# Example usage
title1 = "Understanding Machine Learning"
title2 = "Introduction to Deep Learning"
distance = calculate_euclidean_distance(title1, title2)
print(f"Euclidean Distance: {distance}")



In [ ]:
# add a column to the dataframe similarities that contains the euclidean distance between the embeddings of the articles in each pair
df['euclidean_distance'] = df['pair'].apply(lambda x: calculate_euclidean_distance(x[0], x[1]))

In [ ]:
# save the dataframe to a csv file
df.to_csv('article_similarity.csv', index=False)